In [30]:
import numpy as np
from keras.datasets import mnist
import pandas as pd

# Importing the dataset and preprocessing to get it in the desired shape
(X_train, y_train), (X_test, y_test) = mnist.load_data()

#Scaling the values to scale the data down
X_train = X_train/255
X_test = X_test/255


temp = X_train.reshape(60000,28*28).T
temp = temp.reshape(28,28,1,60000)
temp = temp[:,:,:,:1000]
ytemp = y_train[:1000]

temp1 = X_test.reshape(10000,28*28).T
temp1 = temp1.reshape(28,28,1,10000)
temp1 = temp1[:,:,:,:1000]
ytemp1 = y_test[:1000]

In [31]:
# Flatten function is used to convert a convoluted matrix into a single array
# to apply in FC layer
def flatten(X):
    shape_x=X.shape[0]
    shape_y=X.shape[1]
    shape_z=X.shape[2]
    m=X.shape[3]
    flattened_vec = X.reshape(shape_x*shape_y*shape_z,m)
    return flattened_vec,shape_x,shape_y,shape_z,m

In [32]:
# Zero-pad function is to do padding of the input matrix to minimize data loss
def zero_pad(X, pad):
    
    X_pad = np.pad(X, ((pad,pad),(pad,pad),(0,0),(0,0)), 'constant', constant_values=0)
    return X_pad

In [33]:
# Softmax function is used in the final step
def softmax(x):
    softm = np.exp(x)/np.sum(np.exp(x),axis=1,keepdims=True)
    return softm

In [34]:
# Relu function is used as activation function in the convolution layers
def relu(x):
    y= np.maximum(0,x)
    return y

In [35]:
# The gradient of relu function
def relugradient(dact,act):
    dhidden = dact
    dhidden[act<=0]=0
    return dhidden 

In [53]:
def batchnorm_forward(x, gamma, beta, eps):

    N, D = x.shape
    #step1: calculate mean
    mu = 1./N * np.sum(x, axis = 0)
    #step2: subtract mean vector of every trainings example
    xmu = x - mu
    #step3: following the lower branch - calculation denominator
    sq = xmu ** 2
    #step4: calculate variance
    var = 1./N * np.sum(sq, axis = 0)
    #step5: add eps for numerical stability, then sqrt
    sqrtvar = np.sqrt(var + eps)
    #step6: invert sqrtwar
    ivar = 1./sqrtvar
    #step7: execute normalization
    xhat = xmu * ivar
    #step8: the two transformation steps
    gammax = gamma * xhat
    #step9
    out = gammax + beta
    #store intermediate
    cache = (xhat,gamma,xmu,ivar,sqrtvar,var,eps)
    return out, cache


In [54]:
def convbatchnorm_forward(X, gamma, beta, eps = 1e-5):
    H, W, C, N = X.shape
        # mini-batch mean
    mu = np.mean(X, axis=(0, 1, 3))
    xmu = X -mu.reshape(1,1,C,1)
    sq = xmu**2
        # mini-batch variance
    var =(1/(H*W*N)) *np.sum(sq, axis=(0, 1, 3)).reshape((1,1,C,1))
    sqrtvar = np.sqrt(var+eps)
    ivar = 1./sqrtvar
    xhat = xmu * ivar
    #print(gamma.shape)
    gammax = gamma.reshape(1,1,C,1) * xhat
    out = gammax + beta.reshape(1,1,C,1)
    cache = (xhat,gamma,xmu,ivar,sqrtvar,var,eps)
    return out , cache

In [55]:
def batchnorm_backward(dout, cache):
      #unfold the variables stored in cache
    xhat,gamma,xmu,ivar,sqrtvar,var,eps = cache
      #get the dimensions of the input/output
    N,D = dout.shape
      #step9
    dbeta = np.sum(dout, axis=0)
    dgammax = dout #not necessary, but more understandable
      #step8
    dgamma = np.sum(dgammax*xhat, axis=0)
    dxhat = dgammax * gamma
  #step7
    divar = np.sum(dxhat*xmu, axis=0)
    dxmu1 = dxhat * ivar
  #step6
    dsqrtvar = -1. /(sqrtvar**2) * divar
  #step5
    dvar = 0.5 * 1. /np.sqrt(var+eps) * dsqrtvar
  #step4
    dsq = 1. /N * np.ones((N,D)) * dvar
  #step3
    dxmu2 = 2 * xmu * dsq
  #step2
    dx1 = (dxmu1 + dxmu2)
    dmu = -1 * np.sum(dxmu1+dxmu2, axis=0)
  #step1
    dx2 = 1. /N * np.ones((N,D)) * dmu
  #step0
    dx = dx1 + dx2
    return dx, dgamma, dbeta

In [56]:
def convbatchnorm_backward(dout, cache):
    #unfold the variables stored in cache
    xhat,gamma,xmu,ivar,sqrtvar,var,eps = cache
    #get the dimensions of the input/output
    H,W,C,N = dout.shape
    #step9
    dbeta = np.sum(dout, axis=(0,1,3)).reshape(1,1,C,1)
    dgammax = dout #not necessary, but more understandable
    #step8
    dgamma = np.sum(dgammax*xhat, axis=(0,1,3)).reshape(1,1,C,1)
    dxhat = dgammax * gamma.reshape(1,1,C,1)
    #step7
    divar = np.sum(dxhat*xmu, axis=(0,1,3)).reshape(1,1,C,1)
    dxmu1 = dxhat * ivar
    #step6
    dsqrtvar = -1. /(sqrtvar**2) * divar
    #step5
    dvar = 0.5 * 1. /np.sqrt(var+eps) * dsqrtvar
    #step4
    dsq = (1. /(H*W*N)) * np.ones((H,W,C,N)) * dvar
    #step3
    dxmu2 = 2 * xmu * dsq
    #step2
    dx1 = (dxmu1 + dxmu2)
    dmu = -1 * np.sum(dxmu1+dxmu2, axis=(0,1,3))
    #step1
    dx2 = (1. /(H*W*N)) * np.ones((H,W,C,N)) * dmu.reshape(1,1,C,1)
    #step0
    dx = dx1 + dx2
    return dx, dgamma, dbeta

In [57]:
# conv_forward function is to apply convolution on the input using filters
def conv_forward(X,W,padding=1,stride=1):       
    p = int(padding)
    if (padding is not 0):
        X = zero_pad(X,p)
        #X = np.pad(X, ((p,p),(p,p),(0,0),(0,0)), 'constant')
    n = int(X.shape[0])
    #print(n)
    m = int(X.shape[3])
   # print(m)
    f = int(W.shape[0])
    #print(f)
    s = int(stride)
    #print(s)
    num_channels=int(X.shape[2])
    num_filters=W.shape[3]
    if (type((n-f)//s) is not int):
        print((n-f)//s)
        print('invalid padding or stride')
        return
    #output_size = int((n-f+2*p)/s) +1
    output_size = int((n-f)/s) + 1
    #print(output_size)
    Z = np.zeros((output_size,output_size,num_filters,m))
    for k in range(num_filters):
        for i in range(output_size):
            for j in range(output_size):
                Z[i,j,k,:]=np.sum(X[i*s:(i*s+f),j*s:(j*s+f),:,:]*W[:,:,:,k].reshape(f,f,num_channels,-1),axis=(0,1,2))
                
    return Z


In [58]:
def cnn_pool_forward(X,f=2,stride=2):       
    n = int(X.shape[0])
    m = int(X.shape[3])
    s = int(stride)
    num_filters=int(X.shape[2])
    output_size = int((n-f)/s) +1
    Z = np.zeros((output_size,output_size,num_filters,m))
    for k in range(num_filters):
        for i in range(output_size):
            for j in range(output_size):
                Z[i,j,k,:]=np.max(X[i*s:(i*s+f),j*s:(j*s+f),k,:],axis=(0,1))                
    return Z

In [59]:
def conv_backward(dZ,W,X,padding=1,stride=1):
   
 #    The backward computation for a convolution function
#    
#    Arguments:
#    dZ -- gradient of the cost with respect to output of the conv layer (Z), numpy array of shape (n_H, n_W) 
#    W -- weights of the conv layer
#    X -- input of the conv layer
#    
#    Returns:
#    dX -- gradient of the cost with respect to input of the conv layer (X), numpy array of shape (n_H_prev, n_W_prev) 
#    dfilter -- gradient of the cost with respect to the weights of the conv layer (W), numpy array of shape (f,f) 

    # Retrieving dimensions from W's shape
    (f, f, num_channels,num_filters) = W.shape
    
    # Retrieving dimensions from dZ's shape
    (n_H, n_W,_,_) = dZ.shape
    
    s = stride
    pad = padding
    # Initializing dX, dW with the correct shapes
    dfilter = np.zeros(W.shape)
    dX = np.zeros(X.shape)
    m = X.shape[3]
    
    # Pad X and dX
    X_pad = zero_pad(X, pad)
    dX_pad = zero_pad(dX, pad)
    
    
    # Looping over vertical(h) and horizontal(w) axis of the output
    for k in range(num_filters):
        # To eliminate the looping over each training example,
        # we create a 5-dim filter matrix where the filter matrix is repeated along the 5th dim
        shapedfilter=np.tile(W[:,:,:,k].reshape(f,f,num_channels,1),(1,1,1,1,m))[0]
        for h in range(n_H):
            for w in range(n_W):
                dX_pad[h*s:h*s+f, w*s:w*s+f,:,:] += shapedfilter * dZ[h,w,k,:]
                dfilter[:,:,:,k] += np.sum(X_pad[h*s:h*s+f, w*s:w*s+f,:,:] * dZ[h,w,k,:],axis=3)
                
                #print(X[h:h+f, w:w+f,:].shape)
    dX = dX[pad:-pad, pad:-pad, :,:]
    return dfilter,dX

In [60]:
def cnn_pool_backward(dA_pooled,A_act,f=2,stride=2):
    dA_act = np.zeros(A_act.shape)
    s = stride
    (n_H,n_W,n_C,_)=dA_pooled.shape
    dA_pooled_reshape=np.repeat(np.repeat(dA_pooled,[f],axis=1),[f],axis=0)
    for c in range(n_C):
        for h in range(n_H):
            for w in range(n_W):
                #print(h,w)
                mask = (A_act[h*s:h*s+f,w*s:w*s+f,c,:]==np.max(A_act[h*s:h*s+f,w*s:w*s+f,c,:],axis=(0,1)))
                #print(mask[:,:,0])
                dA_act[h*s:h*s+f,w*s:w*s+f,c,:]=dA_pooled_reshape[h*s:h*s+f,w*s:w*s+f,c,:]*mask                   
    return dA_act

In [61]:
def calculate_loss(y,y_prob):
    loss_train=0
    m = y.shape[0]
    for i in np.arange(m):
        loss_train = loss_train+ (-np.log(y_prob[i,y[i]]))
    loss_train = loss_train/m
    return loss_train

In [62]:
def accuracy_score(y,y_prob):
    pred=pd.DataFrame(y_prob).idxmax(axis=1)
    return np.mean(pred==y)

In [63]:
def tell_size(temp,ytemp,myfilter1,myfilter2):
    feat_map = conv_forward(temp,myfilter1,stride=1)
    feat_map_act = relu(feat_map)
    feat_map_pool = cnn_pool_forward(feat_map_act,f=2,stride=2)
    feat_map2= conv_forward(feat_map_pool,myfilter2,stride=1)
    feat_map2_act=relu(feat_map2)
    flattened_vec,shape_x,shape_y,shape_z,m = flatten(feat_map2_act)
    return flattened_vec.shape[0]

In [64]:
def forward_propagate(temp,ytemp,parameters,fc1_neurons,num_class):
        (myfilter1,myfilter2,W1,W2,b2,conv1gam,conv2gam,conv1bet,conv2bet,gam,bet)=parameters
        feat_map = conv_forward(temp,myfilter1)
        feat_map_bn,conv1_cache = convbatchnorm_forward(feat_map,conv1gam,conv1bet)
        feat_map_act = relu(feat_map_bn)
        feat_map_pool = cnn_pool_forward(feat_map_act,f=2,stride=2)
        feat_map2= conv_forward(feat_map_pool,myfilter2)
        feat_map2_bn,conv2_cache = convbatchnorm_forward(feat_map2,conv2gam,conv2bet)
        feat_map2_act=relu(feat_map2_bn)
        flattened_vec,shape_x,shape_y,shape_z,m = flatten(feat_map2_act)
        shape = (shape_x,shape_y,shape_z,m)
        fc1 = np.dot(flattened_vec.T,W1)
        fc1_bn,fc_cache=batchnorm_forward(fc1, gamma=gam, beta=bet, eps = 1e-5)
        fc1_act = relu(fc1_bn)
        output = np.dot(fc1_act,W2)+b2
        prob   = softmax(output)
        loss=calculate_loss(ytemp,prob)
        accuracy=accuracy_score(ytemp,prob)
        cache =(feat_map_act,feat_map_pool,feat_map2_act,flattened_vec,shape,
                fc1_act,prob,myfilter1,myfilter2,W1,W2,conv2_cache,conv1_cache,fc_cache)
        
        return loss,accuracy,cache

In [65]:
def backward_propagate(temp,ytemp,cache):
        (feat_map_act,feat_map_pool,feat_map2_act,flattened_vec,shape,
                fc1_act,prob,myfilter1,myfilter2,W1,W2,conv2_cache,conv1_cache,fc_cache)=cache
        shape_x,shape_y,shape_z,m=shape
        doutput = np.copy(prob)
        m = ytemp.shape[0]
        for i in np.arange(m):
            doutput[i,ytemp[i]] = doutput[i,ytemp[i]]-1
        doutput = doutput/m
        dW2 = np.dot(fc1_act.T,doutput)
        db2 = np.sum(doutput, axis=0, keepdims=True)
        dfc1_act = np.dot(doutput,W2.T)
        dfc1_bn = relugradient(dfc1_act,fc1_act)
        dfc1,dgam,dbet = batchnorm_backward(dfc1_bn, fc_cache)
        dW1 = np.dot(flattened_vec,dfc1)
        #db3 shape (1,outputsize)
#        db = np.sum(dfc1, axis=0, keepdims=True)
        dflattened_vec = np.dot(dfc1,W1.T).T
        dfeat_map2_act=dflattened_vec.reshape(shape_x,shape_y,shape_z,m)
        dfeat_map2_bn = relugradient(dfeat_map2_act,feat_map2_act)   
        dfeat_map2,dconv2gam,dconv2bet = convbatchnorm_backward(dfeat_map2_bn,conv2_cache)
        #print(dconv2gam.shape)
        dfilt2,dfeat_map_pooled = conv_backward(dfeat_map2,myfilter2,feat_map_pool)
        dfeat_map_act=cnn_pool_backward(dfeat_map_pooled,feat_map_act,f=2,stride=2)
        dfeat_map_bn = relugradient(dfeat_map_act,feat_map_act)
        dfeat_map,dconv1gam,dconv1bet = convbatchnorm_backward(dfeat_map_bn,conv1_cache)
        #print(dconv1gam.shape)
        dfilt1,dX = conv_backward(dfeat_map,myfilter1,temp)
        
        gradients =(dfilt1,dfilt2,dW1,dW2,db2,dconv1gam,dconv2gam,dconv1bet,dconv2bet,dgam,dbet)
        return gradients

In [66]:
def gradient_descent_optimizer(parameters,gradients,alpha):
        (myfilter1,myfilter2,W1,W2,b2,conv1gam,conv2gam,conv1bet,conv2bet,gam,bet)=parameters
        (dfilt1,dfilt2,dW1,dW2,db2,dconv1gam,dconv2gam,dconv1bet,dconv2bet,dgam,dbet)=gradients
        myfilter1 = myfilter1 -alpha*dfilt1
        myfilter2 = myfilter2 -alpha*dfilt2
        W1 = W1 - alpha * dW1
        W2 = W2 - alpha * dW2
        b2 = b2 - alpha * db2
        conv1gam = conv1gam -alpha * dconv1gam
        conv1bet = conv1bet -alpha * dconv1bet
        conv2gam = conv2gam -alpha * dconv2gam
        conv2bet = conv2bet -alpha * dconv2bet        
        gam = gam - alpha * dgam
        bet = bet - alpha * dbet
        updated_parameters = (myfilter1,myfilter2,W1,W2,b2,conv1gam,conv2gam,conv1bet,conv2bet,gam,bet)
        return updated_parameters

In [67]:

def train_CNN(temp,ytemp,temp1,ytemp1,myfilter1,myfilter2,fc1_neurons,num_class,alpha=0.05,iterations=50):
    Loss1=[]
    Accuracy1=[]
    Loss2=[]
    Accuracy2=[]
    alpha=alpha
    iterations=iterations
    gam=1
    bet=0
    conv1gam=np.ones(myfilter1.shape[3]).reshape(1,1,myfilter1.shape[3],1)
    conv2gam=np.ones(myfilter2.shape[3]).reshape(1,1,myfilter2.shape[3],1)
    conv1bet=np.zeros(myfilter1.shape[3]).reshape(1,1,myfilter1.shape[3],1)
    conv2bet=np.zeros(myfilter2.shape[3]).reshape(1,1,myfilter2.shape[3],1)
    size=tell_size(temp,ytemp,myfilter1,myfilter2)
    W1 = np.random.normal(0,0.1,(size,fc1_neurons))
    W2 = np.random.normal(0,0.1,(fc1_neurons,num_class))
    b2 = np.zeros((1,num_class))       
    parameters = (myfilter1,myfilter2,W1,W2,b2,conv1gam,conv2gam,conv1bet,conv2bet,gam,bet)
    for i in range(iterations):
        loss1,accuracy1,cache_train=forward_propagate(temp,ytemp,parameters,fc1_neurons,num_class)
        loss2,accuracy2,cache_test=forward_propagate(temp1,ytemp1,parameters,fc1_neurons,num_class)
        
        Loss1.append(loss1)
        Accuracy1.append(accuracy1)
        Loss2.append(loss2)
        Accuracy2.append(accuracy2)
        
        print(str(i)+":Train Loss:"+str(loss1)+" | Train Accuracy: "+str(accuracy1))
        print(str(i)+":Test Loss:"+str(loss2)+" | Test Accuracy: "+str(accuracy2))
        
        gradients=backward_propagate(temp,ytemp,cache_train)
        parameters=gradient_descent_optimizer(parameters,gradients,alpha)
    return Loss1,Accuracy1,Loss2,Accuracy2

In [68]:
myfilter1 = np.random.normal(0,0.1,9*16).reshape(3,3,1,16)
myfilter2 = np.random.normal(0,0.1,16*16*18).reshape(4,4,16,18)

trainlossbn,trainaccuracybn,testlossbn,testaccuracybn=train_CNN(temp,ytemp,temp1,ytemp1,myfilter1,myfilter2,fc1_neurons=20,num_class=10,alpha=0.5,iterations=50)    
pd.Series(trainlossbn).plot()    
pd.Series(trainaccuracybn).plot()
pd.Series(testlossbn).plot()    
pd.Series(testaccuracybn).plot()

0: Loss:2.3955485204261833 Accuracy: 0.058
0:Test Loss:2.376046717953194 | Test Accuracy: 0.073
1: Loss:2.218020610898184 Accuracy: 0.177
1:Test Loss:2.2437489471572722 | Test Accuracy: 0.139
2: Loss:2.0853944244241904 Accuracy: 0.355
2:Test Loss:2.1420160959253978 | Test Accuracy: 0.256
3: Loss:1.9685825035794149 Accuracy: 0.485
3:Test Loss:2.0487929265749547 | Test Accuracy: 0.371
4: Loss:1.853142717533198 Accuracy: 0.569
4:Test Loss:1.954309750412736 | Test Accuracy: 0.453
5: Loss:1.7346929500987083 Accuracy: 0.627
5:Test Loss:1.8556369715406418 | Test Accuracy: 0.504
6: Loss:1.6131935688832308 Accuracy: 0.657
6:Test Loss:1.7529607317409532 | Test Accuracy: 0.538
7: Loss:1.4909081595719682 Accuracy: 0.696
7:Test Loss:1.6471526653589676 | Test Accuracy: 0.561
8: Loss:1.3717372654291187 Accuracy: 0.729
8:Test Loss:1.5419635825689355 | Test Accuracy: 0.593
9: Loss:1.2587098365548828 Accuracy: 0.764
9:Test Loss:1.4379935959259904 | Test Accuracy: 0.623
10: Loss:1.1534356334596598 Accura